In [ ]:
# GladCam, TF Serving
# SageMaker BiT, BYOS, Debugger

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import configparser
import os
import sys
import wandb
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from albumentations import (
    Compose, 
    ShiftScaleRotate,
)
from sklearn.model_selection import ShuffleSplit

sys.path.append("..")
from utils.data_utils import (
    identity_func,
    load_tfrecord_dataset,
    mixup_dataset,
)

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")
np.random.seed(42)

In [ ]:
config = configparser.ConfigParser()
_ = config.read(os.path.join("..", "conf", "config.ini"))

project_name = config["project"]["project_name"]
run_name = config["project"]["run_name"]
raw_data_path = config["project"]["raw_data_path"]
plabeled_data_path = config["project"]["plabeled_data_path"]

model_url = config["model"]["model_url"]
fc_size = eval(config["model"]["fc_size"])
img_size = eval(config["model"]["img_size"])
n_epochs = eval(config["model"]["n_epochs"])
batch_size = eval(config["model"]["batch_size"])
initial_learning_rate = eval(config["model"]["initial_learning_rate"])
first_decay_steps = eval(config["model"]["first_decay_steps"])
use_adamw = eval(config["model"]["use_adamw"])
use_mixup = eval(config["model"]["use_mixup"])
use_swa = eval(config["model"]["use_swa"])
label_smoothing = eval(config["model"]["label_smoothing"])

label_names = {
    "c0": "safe driving",
    "c1": "texting - right",
    "c2": "talking on the phone - right",
    "c3": "texting - left",
    "c4": "talking on the phone - left",
    "c5": "operating the radio",
    "c6": "drinking",
    "c7": "reaching behind",
    "c8": "hair and makeup",
    "c9": "talking to passenger",
}

labels = list(label_names.keys())
num_classes = len(label_names)

In [ ]:
run = wandb.init(project=project_name, reinit=False)

if len(run_name) > 0:
    wandb.run.name = run_name
    wandb.run.save()
    
wandb.config = {
    "model_url": model_url,
    "fc_size": fc_size,
    "img_size": img_size,
    "n_epochs": n_epochs,
    "batch_size": batch_size,
    "initial_learning_rate": initial_learning_rate,
    "first_decay_steps": first_decay_steps,
    "use_adamw": use_adamw,
    "use_mixup": use_mixup,
    "use_swa": use_swa,
    "label_smoothing": label_smoothing,
}

## The Input Data Pipeline Configuration with *Dataset* API

In [ ]:
train_plabeled_data_path = os.path.join(plabeled_data_path, "imgs", "train")
test_plabeled_data_path = os.path.join(plabeled_data_path, "imgs", "test")

n_train_examples = len(
    tf.io.gfile.glob(os.path.join(train_plabeled_data_path, "*", "*.jpg"))
)
test_pseudo_labeled_ids = [
    os.path.basename(img_path)
    for img_path in np.sort(
        tf.io.gfile.glob(os.path.join(test_plabeled_data_path, "*.jpg"))
    )
]
n_test_examples = len(test_pseudo_labeled_ids)

In [ ]:
train_tfrec_paths = tf.io.gfile.glob(
    os.path.join(plabeled_data_path, "tfrec", "train", "*.tfrec")
)
test_tfrec_paths = np.sort(
    tf.io.gfile.glob(os.path.join(plabeled_data_path, "tfrec", "test", "*.tfrec"))
).tolist()

train_transforms = Compose(
    [
        ShiftScaleRotate(
            rotate_limit=(-20, 20),
            scale_limit=(0.0, 0.2),
            shift_limit_x=(-0.0625, 0.0625),
            shift_limit_y=(-0.046875, 0.046875),
            p=1.0,
        ),
    ]
)
valid_transforms = train_transforms
test_transforms = identity_func

## Building and Compiling the Model

In [ ]:
def get_model(verbose=False):
    model = tf.keras.Sequential(
        [
            hub.KerasLayer(
                model_url, trainable=False, input_shape=(img_size, img_size, 3)
            ),
            tf.keras.layers.Dense(fc_size, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(fc_size, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    if verbose:
        print(model.summary())

    # For tensorflow 2.5 or later, use tf.keras.optimizers.schedules.CosineDecayRestarts.
    lr_decayed_fn = tf.keras.experimental.CosineDecayRestarts(
        initial_learning_rate, first_decay_steps
    )
    optimizer = (
        tfa.optimizers.AdamW(lr_decayed_fn)
        if use_adamw
        else tfa.optimizers.RectifiedAdam(lr_decayed_fn)
    )
    if use_swa:
        optimizer = tfa.optimizers.SWA(optimizer, start_averaging=0, average_period=10)

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing),
        metrics=["acc"],
    )
    
    return model

## Model Training and Prediction

In [ ]:
n = n_train_splits if n_train_splits > 1 else n_train_splits + 1
train_steps_per_epoch = round(
    n_train_examples * (n - 1) / n / batch_size
)
valid_steps_per_epoch = round(n_train_examples / n / batch_size)

test_dataset = load_tfrecord_dataset(
    test_tfrec_paths,
    img_size,
    test_transforms,
    1,
    shuffle=False,
    num_classes=num_classes,
    is_prediction=True,
)

rs = ShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
split = rs.split(range(len(train_tfrec_paths)))
train_index, valid_index = next(split)

prediction_path = os.path.join("..", "predictions")
saved_model_path = os.path.join("..", "saved_models")

os.makedirs(saved_model_path, exist_ok=True)

In [ ]:
with get_elapsed_time():
    one_train_dataset = load_tfrecord_dataset(
        np.array(train_tfrec_paths)[train_index],
        img_size,
        train_transforms,
        batch_size,
        num_classes=num_classes,
    )

    if use_mixup:
        oth_train_dataset = load_tfrecord_dataset(
            np.array(train_tfrec_paths)[train_index],
            img_size,
            train_transforms,
            batch_size,
            num_classes=num_classes,
        )
        zipped = tf.data.Dataset.zip((one_train_dataset, oth_train_dataset))
        train_dataset = zipped.map(
            lambda x, y: mixup_dataset(x, y, alpha=0.2),
            num_parallel_calls=tf.data.experimental.AUTOTUNE,
        )
    else:
        train_dataset = one_train_dataset

    valid_dataset = load_tfrecord_dataset(
        np.array(train_tfrec_paths)[valid_index],
        img_size,
        valid_transforms,
        batch_size,
        num_classes=num_classes,
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(model_path, f"final_model.h5"),
            monitor="val_loss",
        ),
        WandbCallback(),
    ]

    model = get_model()

    model.fit(
        train_dataset,
        epochs=n_epochs,
        steps_per_epoch=train_steps_per_epoch,
        validation_data=valid_dataset,
        validation_steps=valid_steps_per_epoch,
        callbacks=callbacks,
    )

    prediction = model.predict(
        test_dataset,
        steps=n_test_examples,
        use_multiprocessing=True,
    )

    tf.keras.models.save_model(
        model,
        saved_model_path,
        overwrite=True,
        include_optimizer=True,
        save_format=None,
        signatures=None,
        options=None,
    )

curr_result = pd.DataFrame(prediction, columns=labels, index=test_pseudo_labeled_ids)
prev_result = pd.read_csv(os.path.join(prediction_path, "submission.csv"), index_col=0)
result = pd.concat(
    [curr_result, prev_result.loc[prev_result.index.difference(curr_result.index)]]
)

result.to_csv(os.path.join(prediction_path, "final_submission.csv"))

print("The model training and prediction tasks have been successfully completed.")

In [ ]:
n_samples = 20


lowest_confident_ids = result.max(axis=1).sort_values()[:n_samples].index.tolist()
lowest_confident_result = result.loc[lowest_confident_ids].values
columns = ["image_id", "image", "label", "proba"]

data = []
for each_id, label, proba in zip(
    lowest_confident_ids,
    lowest_confident_result.argmax(axis=1),
    lowest_confident_result.max(axis=1),
):
    data.append(
        [
            each_id,
            wandb.Image(os.path.join(test_raw_data_path, each_id)),
            label_names[f"c{label}"],
            round(proba, 6),
        ]
    )

lowest_confident_img_table = wandb.Table(data=data, columns=columns)
wandb.log({"lowest_confident_img_table": lowest_confident_img_table})
run.finish()

In [ ]:
# serving test